In [41]:
!pip install gradio langchain langchain_community langchain_milvus transformers pymilvus


In [42]:
!pip install replicate

In [43]:
!pip install sentence-transformers

In [44]:
import os
import gradio as gr
import tempfile
from langchain_milvus import Milvus
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# 🔑 Replicate token and model setup
os.environ['REPLICATE_API_TOKEN'] = "r8_ZTwLnTd6CArGhOR1SIWnxHn2c8XvfWh2ni6YP"
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(model=model_path, replicate_api_token=os.environ['REPLICATE_API_TOKEN'])

# 📦 Temporary Milvus DB
db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": db_file},
    auto_id=True,
    index_params={"index_type": "AUTOINDEX"},
)

# 📄 Local Research Data
filename = "research_data.txt"
with open(filename, "w") as f:
    f.write("""
Research Problem Statement:
This project aims to build a research assistant that helps users explore academic papers, literature, and resources related to our research topic.

How it works:
- User uploads or accesses stored research papers, abstracts, and summaries.
- The AI retrieves relevant information from stored documents.
- It answers questions in an easy-to-understand way, citing relevant sources if available.

Example Topics:
- Problem statement explanation
- Literature review summaries
- Research methodologies
- Advantages and disadvantages of approaches
- Links to download papers (IEEE, ResearchGate, etc.)
- Key findings and discussions

Usage Instructions:
Ask questions like:
- "Summarize the problem statement"
- "Give literature review on AI in healthcare"
- "List advantages and disadvantages of blockchain in supply chain"
""")

# Load & Split Content
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=tokenizer.model_max_length // 2,
    chunk_overlap=0,
)
texts = splitter.split_documents(documents)
for i, doc in enumerate(texts):
    doc.metadata["doc_id"] = i + 1
vector_db.add_documents(texts)

# 🧠 Research Assistant Prompt
template = """
You are a friendly but highly knowledgeable Research Assistant.
If the user greets you, greet them back politely and warmly.
If the question is research-related, provide a structured academic answer:
- Summary of the concept
- Key findings or literature review
- Advantages & disadvantages
- Practical applications
Always try to give clear, reliable information.
User Question: {question}
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

# 🎯 Main QnA Function with Fallback
def ask_research_agent(query):
    try:
        # Greeting check
        if query.strip().lower() in ["hi", "hello", "hey", "hii"]:
            return "Hi there! 👋 I’m your AI Research Assistant. Ask me anything about your research project or related topics."

        # Retrieve from Milvus
        retriever = vector_db.as_retriever(search_kwargs={"k": 2})
        retrieved_docs = retriever.get_relevant_documents(query)

        # If no relevant documents → fallback to direct LLM
        if not retrieved_docs or all(len(doc.page_content.strip()) == 0 for doc in retrieved_docs):
            return llm_chain.run({"question": query})

        # If relevant docs found → use them for context
        combined_content = "\n".join([doc.page_content for doc in retrieved_docs])
        return llm_chain.run({"question": query + "\n\nContext:\n" + combined_content})

    except Exception as e:
        return f"❌ Error: {str(e)}"

# 💻 Gradio Interface
iface = gr.Interface(
    fn=ask_research_agent,
    inputs=gr.Textbox(label="Ask a question about research", placeholder="e.g. Explain AI in healthcare"),
    outputs=gr.Textbox(label="Answer"),
    title="AI Research Assistant",
    description="Ask anything about your research or related academic topics.",
    theme="default"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cf0efb494eefc44425.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
